In [1]:
!nvidia-smi

Tue Jan  6 06:13:58 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           On  |   00000000:06:00.0 Off |                    0 |
| N/A   29C    P0             54W /  300W |   31166MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install flwr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.1/727.1 kB 14.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 3.4 MB/s  0:00:01m0:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 21.6 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: grpcio━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  2/10 [protobuf]
    Found existing installation: grpcio 1.73.1━━━━━━━━━━━━━━━━  2/10 [protobuf]
    Uninstalling grpcio-1.73.1:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  2/10 [protobuf]
      Successfully uninstalled grpcio-1.73.1━━━━━━━━━━━━━━━━━━  2/10 [protobuf]
  Attempting uninstall: click╺━━━━━━━━━━━━━━━━━━━  5/10 [grpcio]
    Found existing installation: click 8.2.1━━━━━━━━━━━━━━━━━━  5/10 [grpcio]
    Uninstalling click-8.2.1:╺━━━━━━━━━━━━━━━━━━━  5/10 [grpcio]
      Successfully uninstalled click-8.2.1m━━━━━━━━━━━━━━━━

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

print(f"🔧 PyTorch version: {torch.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Initial VRAM: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")

2026-01-06 06:15:36.540086: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


🔧 PyTorch version: 2.9.0+cu128
🎮 CUDA available: True
🎯 GPU: Tesla V100-SXM2-32GB
💾 Initial VRAM: 0.00 GB


In [5]:
!nvidia-smi

Tue Jan  6 06:15:40 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           On  |   00000000:06:00.0 Off |                    0 |
| N/A   29C    P0             54W /  300W |   31166MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:

dataset_path = "../data/processed/hospital_A/dataset.jsonl"

def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]
train_data = load_jsonl(dataset_path)
print(f"\n✅ Loaded {len(train_data)} samples from Hospital A")
print(f"\n📝 Sample entry:")
sample = train_data[0]
print(f"  - Text length: {len(sample['text'])} chars")
print(f"  - Token count: {sample['token_length']}")
print(f"  - Preview: {sample['text'][:200]}...")



✅ Loaded 4520 samples from Hospital A

📝 Sample entry:
  - Text length: 1276 chars
  - Token count: 297
  - Preview: User: What is the primary property lost by alveolar Type II Pneumocytes in culture Assistant: Surfactant production is important in maintaining alveolar function both in vivo and in vitro, but surfact...


In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

print("🔄 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("🔄 Loading model with 4-bit quantization on GPU 3...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)

# Prepare for k-bit training (gradient checkpointing)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

if torch.cuda.is_available():
    print(f"💾 VRAM after model load: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
print("✅ Model loaded successfully on GPU 3")


🔄 Loading tokenizer...
🔄 Loading model with 4-bit quantization on GPU 3...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

💾 VRAM after model load: 0.80 GB
✅ Model loaded successfully on GPU 3


In [10]:
lora_config = LoraConfig(
    r=8,                           # LoRA rank
    lora_alpha=16,                 # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Attention modules
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"✅ LoRA adapters applied")
print(f"📊 Trainable params: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
print(f"📊 Total params: {total_params:,}")

✅ LoRA adapters applied
📊 Trainable params: 3,407,872 (0.09%)
📊 Total params: 3,755,479,040


In [ ]:
# Load small subset for testing
print("Loading dataset...")
dataset = load_dataset("AI-MO/ai-medical-dataset")

# Use small subset for quick test
train_data = dataset['train'].select(range(100))
print(f"✓ Loaded {len(train_data)} samples")

In [ ]:
# Preprocess dataset
print("Preprocessing...")
processed_data = preprocess_dataset(train_data, tokenizer, max_length=512)
print("✓ Dataset preprocessed")

In [ ]:
# Train
print("Starting training...")
trainer = train_local(
    model=model,
    train_dataset=processed_data,
    output_dir="../output-models/test-lora"
)
print("✓ Training complete!")

In [ ]:
# Test inference
from src.model.inference import generate_response

test_question = "What causes type 2 diabetes?"
response = generate_response(model, tokenizer, test_question)

print("\n" + "=" * 50)
print(f"Question: {test_question}")
print("=" * 50)
print(f"Response: {response}")

In [ ]:
# Check memory usage
print(f"\nAllocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")